In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm

In [8]:
# 브라우저 옵션 설정 (headless 모드로 실행)
# options = Options()
# options.headless = True

# 이 모드를 사용하면 실제 브라우저 창이 뜨지 않아 시스템 리소스가 절약되고, 스크립트 실행이 더 빠르게 진행됨

In [9]:
# 크롬 드라이버 경로 설정
# service = Service('/path/to/chromedriver')  # 크롬 드라이버 경로를 설정하세요.

In [12]:
# 각 제품군에 대한 URL 정의
urls = {
    "TV": "https://www.lg.com/in/tv-soundbars/all-tv-soundbars/?ec_model_status_code=ACTIVE",
    "세탁기": "https://www.lg.com/in/laundry/all-washing-machines/?ec_model_status_code=ACTIVE",
    "에어컨": "https://www.lg.com/in/air-conditioners/all-air-conditioners/?ec_model_status_code=ACTIVE",
    "냉장고": "https://www.lg.com/in/refrigerators/all-refrigerators/?ec_model_status_code=ACTIVE"
}

In [13]:
# 리뷰 데이터를 저장할 딕셔너리
reviews = {}

# 브라우저 시작
driver = webdriver.Chrome() #options=options
wait = WebDriverWait(driver, 5)

In [ ]:
# 각 제품군에서 데이터를 스크래핑하는 함수
def scrape_reviews(url, category):
    driver.get(url)

    # 'Most Popular'로 정렬
    sort_by_dropdown = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'select[name="sort"]')))
    sort_by_dropdown.click()
    most_popular_option = driver.find_element(By.XPATH, '//option[@value="MOST_POPULAR"]')
    most_popular_option.click()

    while True:
        product_links = driver.find_elements(By.CSS_SELECTOR, 'div.model-list > div.model-item > a')

        for link in tqdm(product_links, desc=f"Processing {category} products"):
            product_url = link.get_attribute('href')
            driver.get(product_url)
            
            # 리뷰 탭 클릭
            reviews_tab = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[contains(text(), "Reviews")]')))
            reviews_tab.click()

            # 'Locale'을 'English (India)'로 설정
            locale_dropdown = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'select#reviewFilterLocale')))
            locale_dropdown.click()
            english_india_option = driver.find_element(By.XPATH, '//option[contains(text(), "English (India)")]')
            english_india_option.click()

            # 'Sort by'를 'Lowest to Highest Rating'으로 설정
            sort_dropdown = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'select#reviewSortOrder')))
            sort_dropdown.click()
            lowest_to_highest_option = driver.find_element(By.XPATH, '//option[@value="rating-asc"]')
            lowest_to_highest_option.click()

            # 리뷰 요소 찾기
            review_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.cmmnt-contents')))
            if category not in reviews:
                reviews[category] = []
            reviews[category].extend([review.text for review in review_elements])

            driver.back()  # 이전 페이지로 돌아가기

        # 다음 페이지가 있는지 확인하고 이동
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, 'a.next')
            next_button.click()
        except:
            break  # 다음 페이지가 없으면 종료

In [ ]:
# 모든 제품군에 대해 리뷰 데이터를 수집
for category, url in urls.items():
    scrape_reviews(url, category)

In [ ]:
# 브라우저 종료
driver.quit()

In [ ]:
# 수집된 리뷰 데이터를 출력
for category, review_list in reviews.items():
    print(f"{category} 리뷰:")
    for review in review_list:
        print(f"- {review}")